In [1]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import pickle
import time
import re
import nltk
import sys
import html
import xml.sax.saxutils as saxutils
from html.parser import HTMLParser
from io import StringIO
import random
import operator

In [2]:
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
def cleaning (text):
    
    #order of lines is important
    
    text=strip_tags(text)
    #text=html.unescape(text)   # stripping or converting html entities 
    #text=saxutils.unescape(text)
 
    
    #converting
    text = re.sub("“|”", '', text)  #before next lines
    text = re.sub("’|′|‘|`", "", text)  #before next lines
    
    #removing tabs and lines
    text=re.sub('\t|\n', ' ', text)
    
    #converting lower_case
    text = text.lower() 
    
    #converting
    text=re.sub('\$|£|€|¥|dollar|dollars|yen|yens|euros', ' money ', text)   # not euro 
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    
    #removing emoji
    text = emoji_pattern.sub(r' ', text) 

    #removing emojis and non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+|,Ä¶',' ', text)  
    
    #removeing http and https (URL)
    text = re.sub(r'(http://|https://)\S+', '', text)
    
    #removing www (URL)
    text=re.sub(r'www\.\S+', '', text)
    
    #removing targets
    text=re.sub('( |^)@\S+', '', text) 

    #removing common expressions
    '''
    text=re.sub("looking forward to|look forward to|make sure|kidding me|\
                |in my opinion|by the way,|as soon as possible|shaking my head|i don't know|I do not know|\
                |in real life|quote of the day|as far as i know|shake my head|\
                |to be honest|in other words|let me know|just kidding|hope that helps|hat tip|\
                |just like that|happy birthday|never mind|well-done|\
                |in my humble opinion|happy new year|you're welcome|you are welcome| \
                |it doesn't matter|it does not matter|i think|i wonder|do you think", ' ', text) 
    '''
    
    #convertings
    text=re.sub("can't", 'cannot', text) # before other n't 
    text=re.sub("can not ", 'cannot ', text)  
    text=re.sub("'ve",' have', text)
    text=re.sub("n't",' not', text)
    text=re.sub("'ll",' will', text)
    #text=re.sub("'d",' would', text)
    text=re.sub("'re",' are', text)
    text=re.sub("i'm",'i am', text)
    text=re.sub("&",' and ', text)
    text=re.sub(" w/ ",' with ', text)
    text=re.sub(" w/i | w/in ",' within ', text)
    text=re.sub(" w/o ",' without ', text)
    text=re.sub(" c/o ",' care of ', text)
    text=re.sub(" h/t ",' hat tip ', text)
    text=re.sub(" b/c ",' because ', text)
    text=re.sub("=",' equals to ', text)
    text=re.sub("=",' = ', text)
    text=re.sub("\+",' + ', text)
    text=re.sub("\+",' plus ', text)
    text=re.sub("united states",'usa', text)
    text=re.sub("united kingdom",'uk', text)
    text=re.sub(" the us ",' usa ', text)
    text=re.sub("start-up|start_up",'startup', text)
    text=re.sub("u\.s\.a", 'usa', text)  #try text=re.sub("u.s.a", 'usa', text) with text=substantially 
    text=re.sub("aka", 'also known as', text)     
    text=re.sub("'"," ' ", text)     
    
    text= re.sub("(\?)+", '? ',text)     
    text= re.sub("(!)+", '! ',text)     
    text= re.sub("(\.\.)+", ' ',text)   

#    text = "".join(lemmatize_sentence(text))
    
    #removing some special charachter  
#    text= re.sub("[\"\+\|\*\?\(\)\/\\\^\[\]``<>\.{}`′’‘';•«»,@:~!\=%&]+", ' ',text)  #except _ -
#    text= re.sub("[\"\“\”\+\-\|\*\?\(\)\/\\\^\[\]\.{}_`′’‘';•«,@:~!\=%&]+", ' ',text) 
    

    
    return text

In [4]:
filter_words=set(['game','games','gamer','ganmeplay','basketball','nba','football','league','play','playing','player','players',])

In [5]:

import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
cur.execute("SELECT user_id, tweet, screen_name from mng_world_tweets_ph2_light ;")
#cur.execute("SELECT user_id, tweet from mng_2019_1000k ")
rows_mng = cur.fetchall()
con.close()

print(len(rows_mng))

pass

Database opened successfully
46502302


In [6]:
'''
df_mng0 = pd.read_csv('mng_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False,warn_bad_lines=False)
#df_mng0 = pd.read_csv('/archives1/Datasets/TweetsWorld/mng_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False,warn_bad_lines=False)
print(df_mng0.shape)
print(df_mng0.columns)
print(df_mng0.head()) # Preview the first 5 lines of the loaded data 

rows_mng=list(df_mng0[['user_id', 'tweet']].itertuples(index=False, name=None)) #rows_mng0
#rows_mng= list(zip(df_mng0.user_id, df_mng.tweet))
#rows_mng=df_mng0[['user_id','tweet']].apply(tuple, axis=1) 
#df_mng = df_mng0[['user_id', 'tweet','tweet_created_at','location_profile']]

del df_mng0

print(len(rows_mng)) #rows_mng0
print('memry size of mng:', sys.getsizeof(rows_mng)) #rows_mng0
'''
pass

In [7]:
'''
df_ent0 = pd.read_csv('ent_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False,warn_bad_lines=False)
#df_ent0 = pd.read_csv('/archives1/Datasets/TweetsWorld/ent_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False,warn_bad_lines=False)
print(df_ent0.shape)
print(df_ent0.columns)
print(df_ent0.head()) # Preview the first 5 lines of the loaded data 

rows_ent=list(df_ent0[['user_id', 'tweet']].itertuples(index=False, name=None)) #rows_ent0
#rows_ent= list(zip(df_ent0.user_id, df_ent.tweet))
#rows_ent=df_ent0[['user_id','tweet']].apply(tuple, axis=1) 
#df_ent = df_ent0[['user_id', 'tweet','tweet_created_at','location_profile']]

del df_ent0

print(len(rows_ent)) #rows_ent0
print('memry size of ent:', sys.getsizeof(rows_ent)) #rows_ent0
'''
pass

In [8]:
'''
df_public0 = pd.read_csv('public_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False,warn_bad_lines=False)
#df_public0 = pd.read_csv('/archives1/Datasets/TweetsWorld/public_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False,warn_bad_lines=False)
print(df_public0.shape)
print(df_public0.columns)
print(df_public0.head()) # Preview the first 5 lines of the loaded data 

rows_public=list(df_public0[['user_id', 'tweet']].itertuples(index=False, name=None)) #rows_public0
#rows_public= list(zip(df_public0.user_id, df_public.tweet))
#rows_public=df_public0[['user_id','tweet']].apply(tuple, axis=1) 
#df_public = df_public0[['user_id', 'tweet','tweet_created_at','location_profile']]

del df_public0

print(len(rows_public)) #rows_public0
print('memry size of public:', sys.getsizeof(rows_public)) #rows_public0
'''
pass

In [9]:

import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
cur.execute("SELECT user_id, tweet, screen_name from ent_world_tweets_ph2_light ;")
#cur.execute("SELECT user_id, tweet from ent_2019_1000k ")
rows_ent = cur.fetchall()
con.close()

print(len(rows_ent))

pass

Database opened successfully
47604376


In [10]:

import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
cur.execute("SELECT user_id, tweet, screen_name from public_world_tweets_ph2_light ;")
#cur.execute("SELECT user_id, tweet from public_2019_1000k ")
rows_public = cur.fetchall()
con.close()

print(len(rows_public))

pass

Database opened successfully
72182875


In [13]:
rows_mng[0][0]

'25150538'

In [11]:
rows_mng[0],rows_ent[0],rows_public[0]

(('25150538',
  'RT @Z925FM: @Jessica_Z925 is at @AshleyKokomo until noon today with crazy good deals!!! Come out &amp; shop and snag a prize! Girls Scout Cooki…',
  'Jessica_Z925'),
 ('3087448180',
  "RT @MailOnline: New evidence of China's coronavirus cover-up https://t.co/jbGewDBlLC",
  'janet_cutts'),
 ('1251215371870171136',
  'RT @dopeitsdavey: White ppl think this stay at home Order is bad...imagine being inslaved',
  '__sassyjazzy'))

In [12]:

id_screen_mng=dict()
for i in rows_mng:
    id_=i[0]
    screen=i[2]
    if id_ not in id_screen_mng:
        id_screen_mng[id_]=screen

pass

KeyboardInterrupt: 

In [ ]:
user_dic_mng=dict()
for i in rows_mng:
    tweet=i[1]
    text=strip_tags(tweet)
    text=text.lower()
    words=text.split()
    for word in words:
        if word in filter_words:
            user_id=i[0]
            if user_id not in user_dic_mng:
                user_dic_mng[user_id] = 1
            else:
                user_dic_mng[user_id] +=1

In [ ]:
user_dic_mng = dict(sorted(user_dic_mng.items(), key=operator.itemgetter(1), reverse=True))
user_dic_mng = (sorted(user_dic_mng.items(), key=operator.itemgetter(1), reverse=True))

In [ ]:
user_dic_mng[0],user_dic_mng[0][0],user_dic_mng[5318],id_screen_mng[user_dic_mng[5318][0]]

In [17]:
mng_filter=set()
for id_freq in user_dic_mng:
    if id_freq[1] >= 100 :
        mng_filter.add(int(id_freq[0]))
    else:
        break

In [18]:
len(mng_filter)

5319

In [19]:
with open("mng_filtering.txt", "wb") as fp:   
    pickle.dump(mng_filter, fp , protocol=4)

In [20]:
id_screen_ent=dict()
for i in rows_ent:
    id_=i[0]
    screen=i[2]
    if id_ not in id_screen_ent:
        id_screen_ent[id_]=screen

In [22]:
user_dic_ent=dict()
for i in rows_ent:
    tweet=i[1]
    text=strip_tags(tweet)
    text=text.lower()
    words=text.split()
    for word in words:
        if word in filter_words:
            user_id=i[0]
            if user_id not in user_dic_ent:
                user_dic_ent[user_id] = 1
            else:
                user_dic_ent[user_id] +=1

In [23]:
user_dic_ent = dict(sorted(user_dic_ent.items(), key=operator.itemgetter(1), reverse=True))
user_dic_ent = (sorted(user_dic_ent.items(), key=operator.itemgetter(1), reverse=True))

In [24]:
ent_filter=set()
for id_freq in user_dic_ent:
    if id_freq[1] >= 100 :
        ent_filter.add(id_freq[0])
    else:
        break

In [25]:
len(mng_filter)

5319

In [26]:
with open("ent_filtering.txt", "wb") as fp:   
    pickle.dump(ent_filter, fp , protocol=4)

In [27]:
id_screen_public=dict()
for i in rows_public:
    id_=i[0]
    screen=i[2]
    if id_ not in id_screen_public:
        id_screen_public[id_]=screen

In [28]:
user_dic_public=dict()
for i in rows_public:
    tweet=i[1]
    text=strip_tags(tweet)
    text=text.lower()
    words=text.split()
    for word in words:
        if word in filter_words:
            user_id=i[0]
            if user_id not in user_dic_public:
                user_dic_public[user_id] = 1
            else:
                user_dic_public[user_id] +=1

In [29]:
user_dic_public = dict(sorted(user_dic_public.items(), key=operator.itemgetter(1), reverse=True))
user_dic_public = (sorted(user_dic_public.items(), key=operator.itemgetter(1), reverse=True))

In [30]:
public_filter=set()
for id_freq in user_dic_public:
    if id_freq[1] >= 100 :
        public_filter.add(id_freq[0])
    else:
        break

In [31]:
len(public_filter)

3426

In [32]:
with open("public_filtering.txt", "wb") as fp:   
    pickle.dump(public_filter, fp , protocol=4)

In [ ]:
%reset

In [ ]:
import sys
sys.modules[__name__].__dict__.clear()